Desanonimización

Se dispone de un conjunto de datos con los viajes de los taxis de la ciudad de Nueva York hasta el año 2013. En estos datos se han anonimizado las columnas medallion (es decir, el número de licencia de taxi) y hack_license (es decir, el número de licencia del conductor) utilizando MD5. Los formatos originales de estos dos campos tienen la siguiente forma.

Medallion: De acuerdo a los formatos especificados en http://www.nyc.gov/html/tlc_medallion_ info/html/tlc_lookup.shtml.
Licencia: Números de hasta 6 cifras, que pueden contener ceros a la izquierda o no; o bien 7 cifras comenzando siempre con un 5.

Pregunta 5 Entrega:
Realiza la desanonimización de los datos de Medallion y números de licencia. Sube a Moodle el código (Python) utilizado (incluyendo explicaciones y comentarios) y el dataset desanonimizado (2 ficheros en total).

Para desanonimizar las columnas de medallion (licencia de taxi) y hack_license (licencia de conductor) que han sido transformadas mediante MD5, es fundamental recordar que una función hash es unidireccional, lo que significa que no se puede invertir directamente para recuperar el valor original. Por ello, es necesario generar todas las posibles combinaciones respetando la estructura de cada campo. Luego, se calcula el hash MD5 para cada combinación y se compara con los valores almacenados para identificar a qué combinación corresponde cada hash y, de esa forma descifrar la información original.

A continuación se explican los pasos:

In [1]:
import pandas as pd
import hashlib
import itertools
import string

In [2]:
# cargando datos
data = pd.read_csv('trip_data.csv', sep=',')
print(data.head())

                          medallion                      hack_license  \
0  789E7E64EC7C31A88C1683D3BA163A46  3C0FB2975708DF2E873C6D7C72E7EABE   
1  9C34910D26A3CBBC48D309CEA303A1B7  D05DEF3DD93BBCF5D596476C8C18A138   
2  5BA6F6CB6D998B34BAD3701A118E5419  AA6850948D1690435B1339BA4DCC0AA8   
3  3A00BD8BD96A8EFF2BFE18FE274D8A2D  6687D5368226AFA59F6A7ED87D2952E8   
4  A7E52F824BC1D8DC2535D4C6471E7899  7C653CC679536FAA4C1582017D15023B   

  vendor_id  rate_code  store_and_fwd_flag      pickup_datetime  \
0       VTS          1                 NaN  2013-12-05 22:45:00   
1       VTS          1                 NaN  2013-12-05 22:53:00   
2       VTS          1                 NaN  2013-12-05 22:51:00   
3       VTS          1                 NaN  2013-12-05 22:39:00   
4       VTS          1                 NaN  2013-12-05 22:54:00   

      dropoff_datetime  passenger_count  trip_time_in_secs  trip_distance  \
0  2013-12-05 22:54:00                4                540           2.08   
1  2

In [3]:
#solución licencias
# Se crea un diccionario para almacenar los hashes y los valores originales 
hash_dict_lic = {}

# Se generan posibles licencias
# Números de hasta 6 cifras
possible_licenses = [str(i).zfill(6) for i in range(1000000)]  
# Se agregan numeros de 7 cifras que empiezan con 5
possible_licenses += [str(i) for i in range(5000000, 6000000)] 

print(possible_licenses[:10])

['000000', '000001', '000002', '000003', '000004', '000005', '000006', '000007', '000008', '000009']


In [4]:
# Se generan hash de las posibles licencias y se guardan en el diccionario
for license in possible_licenses:
    license_str = str(license)  # Convertir a string
    lic_bytes = license_str.encode()  # Convertir a bytes
    hash_dict_lic[hashlib.md5(lic_bytes).hexdigest()] = license_str
    
print(list(hash_dict_lic.items())[:10])  #primeros 10 valores

[('670b14728ad9902aecba32e22fa4f6bd', '000000'), ('04fc711301f3c784d66955d98d399afb', '000001'), ('768c1c687efe184ae6dd2420710b8799', '000002'), ('f7a5c99c58103f6b65c451efd0f81826', '000003'), ('75891c215fa472036c240d83dddd8b74', '000004'), ('b69b712f7bd6757ddcda59959c89a2b1', '000005'), ('58b2c53441a9db19e159bec686d685d8', '000006'), ('27701bd8dd141b953b94a5c9a44697c0', '000007'), ('7f7d5f9f3a660f2b09e3aae62a15e29b', '000008'), ('23ff17389acbfd020043268fb49e7048', '000009')]


In [5]:
# nueva columna hack_license_real, desanonimizando
data['hack_license_real'] = data['hack_license'].map(lambda x: hash_dict_lic.get(x.strip().lower(), x))

In [6]:
#solución medallion
#se crea lista de números y letras para combinaciones de medallion
digits = [str(i) for i in range(10)]
letters = list(string.ascii_uppercase)

#se crea conjunto vacio para almacenar combinaciones
combinations = set()

#primer tipo: [dígito, letra, dígito, dígito] 
combinations.update("".join(comb) for comb in itertools.product(digits, letters, digits, digits))

#segundo tipo: [letra, letra, dígito, dígito, dígito]
combinations.update("".join(comb) for comb in itertools.product(letters, letters, digits, digits, digits))

#tercer tipo: [letra, letra, letra, dígito, dígito, dígito]
combinations.update("".join(comb) for comb in itertools.product(letters, letters, letters, digits, digits, digits))

#combinations_list = list(combinations)  #lista para manejo de hash
combinations_list = [str(c) for c in combinations] # Se convierte a lista y asegurar que todo sea texto *
print(combinations_list[:10])  #primeros 10 valores generados

['GWF924', 'EDI395', 'NSV700', 'HAM149', 'XDL238', 'XSB305', 'UOZ367', 'BVM930', 'IRK318', 'TDL679']


In [7]:
# Se crea un diccionario para almacenar los hashes y los valores de las medallions (combinaciones)
hash_dict_med = {}

# Se genera hash de las posibles combinaciones y guardando en el diccionario
for license in combinations_list:
    license_str = str(license)  # Convertir a string *
    lic_bytes = license_str.encode()  # Convertir a bytes
    hash_dict_med[hashlib.md5(lic_bytes).hexdigest()] = license_str
    
print(list(hash_dict_med.items())[:10])  #primeros 10 valores

[('4b9806f2a27390aa21b927cb04ab5b67', 'GWF924'), ('67aed7443d4ce79e43088fc7781f8045', 'EDI395'), ('791b125137a65b022a4d29e17c4af182', 'NSV700'), ('d65f5f382558f92a36ec978a2a180ce4', 'HAM149'), ('520afe17230163e4f9cc076b1b1cd843', 'XDL238'), ('272518a9dc277948e5bd430e85e0a8a7', 'XSB305'), ('6d1325a4d3ccc037585af535b807b9ae', 'UOZ367'), ('cc72c5c8080c82dd5b6655331713608a', 'BVM930'), ('76b234e395825ddd13f5b8b0fcce4bad', 'IRK318'), ('2a026be101315f15031f89034a5b2357', 'TDL679')]


In [8]:
# nueva columna medallion_real, desanonimizando
data['medallion_real'] = data['medallion'].map(lambda x: hash_dict_med.get(x.strip().lower(), x))



In [9]:
# Guardar datos desanonimizados
data.to_csv('trip_data_desanonimizado.csv', index=False)



En el archivo generado se encuentra la tabla de los valores anonimizados y al final desanonimizado. Solo Algunos valores no se pudieron descifrar. 